<a href="https://colab.research.google.com/github/pablex72/time-series-practice/blob/main/ndvi_time_series2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#-1 Agua, estructuras artificiales
#0.1 suelo desnudo
#0.2 vegetacion moderada
#0.5 vegetacion densa
#1
#NDVI(Sentinel) = NIR - RED / NIR + RED = 8 - 4 / 8 + 4
#ENERO A MAYO -- JUNIO A DIC DEL 2019

#SENTINEL 2

function coleccion_filtrada(desde, hasta, coleccion, porcentaje_nubes,geometria){
    return coleccion.filterBounds(geometria).filterDate(desde,hasta)
    .filterMetaData('CLOUDY_PERCENTAGE','less_than',porcentaje_nubes);
}

//Creamos una funcion que agrege una banda "NDVI" a una imagen
//Esta nueva banda se calcula coon la funcion normalizedDifference (con RED y NIR)

var addNDVI = function(image){
    var ndvi = image.normalizedDifference(['B8,B4']).rename('NDVI');
    return image.addBands(ndvi);
}

//Mapeamos para cada una de las imagenes de la coleccino la funcion NDVI
function map_coleccion(coleccion){
    return coleccion.map(addNDVI)
}
                                              //desde    //hasta    // coleccion a filter //%nubes //geometria
var coleccionFiltrada2 = coleccion_filtrada('2019-01-01','2019-05-31', coleccion_imagenes, 20, geometry); // primera mitad del anio
var coleccionFiltrada3 = coleccion_filtrada('2019-06-01','2019-12-31', coleccion_imagenes, 20, geometry); // primera mitad del anio

//Mapeamos la funcion addNDVI para cada una de las imagenes de la coleccion
var coleccion_ndvi = map_coleccion(coleccionFiltrada2);
var coleccion_ndvi2 = map_coleccion(coleccionFiltrada3);

var ndvi_s2 = coleccion_ndvi.select('NDVI').median().clip(geometry);
var ndvi_s3 = coleccion_ndvi2.select('NDVI').median().clip(geometry);

//Agregamos la capa al mapa
//creamos una variable con la paleta de colores que queremos utilizar
//para este caso utilizare esta paleta de colores

var paleta = '';

Map.addLayer(ndvi_s2, {min:-0.1, max: 1, palette: paleta}, ''2019-01-01','2019-05-31'')
Map.addLayer(ndvi_s3, {min:-0.1, max: 1, palette: paleta}, ''2019-06-01','2019-12-31'')

var chart = ui.Chart.image.series({
    imageCollection: coleccion_ndvi.select('NDVI'),//usamos la coleccion de imagenes para la primera epoca del anio 
    region: geometry, //Area con la que trabajaremos
    reducer: ee.Reducer.mean(),
    scale:200
})

